In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('data/보유리스크율_자동차/20210728')
BASE_YYMM = '202012'
BASE_YYYY = BASE_YYMM[:4]

In [5]:
# 데이터 불러오기
자동차_원수_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년연간경과보험료_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_ELP_PRM': float})
자동차_비례출재_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년연간경과보험료_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'P_RN_ELP_PRM': float})
자동차_원수_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년연간손해액_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_LOSS': float})
자동차_비례출재_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년연간손해액_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'P_RN_LOSS': float})
자동차_특약보종별_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_ELP_PRM_1YR': float, 'RN_ELP_PRM_1YR': float})
자동차_특약보종별_직전1년손해액 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_직전1년손해액_{BASE_YYMM}.xlsx',
    dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_LOSS_1YR': float, 'RN_LOSS_1YR': float})
자동차_상품군정보 = pd.read_excel(FILE_PATH / '자동차_상품군정보.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})
# 자동차_특약보종별_경과보험료 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_경과보험료_{BASE_YYMM}.xlsx',
#     dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_ELP_PRM': float, 'RN_ELP_PRM': float})
# 자동차_특약보종별_손해액 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_손해액_{BASE_YYMM}.xlsx',
#     dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_LOSS': float, 'RN_LOSS': float})
# 자동차_특약수수료 = pd.read_excel(FILE_PATH / f'자동차_특약수수료_{202012}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
# 자동차_특약정보 = pd.read_excel(FILE_PATH / f'자동차_특약정보_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
# 자동차_상품정보 = pd.read_excel(FILE_PATH / '자동차_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
# 산업_손해율_변동계수 = pd.read_excel(FILE_PATH / f'산업_손해율_변동계수_{BASE_YYMM}.xlsx')
# 자동차_직전1년마감실적 = pd.read_excel(FILE_PATH / f'자동차_연마감실적_{BASE_YYMM}.xlsx', dtype={'보험종목': str})
# 자동차_1년전1년마감실적 = pd.read_excel(FILE_PATH / f'자동차_연마감실적_{int(BASE_YYMM)-100}.xlsx', dtype={'보험종목': str})
# 자동차_2년전1년마감실적 = pd.read_excel(FILE_PATH / f'자동차_연마감실적_{int(BASE_YYMM)-200}.xlsx', dtype={'보험종목': str})
# 자동차_위험계수 = pd.read_excel(FILE_PATH / f'자동차_위험계수_{202012}.xlsx') = pd.read_excel(FILE_PATH / '자동차_상품군정보.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})

In [6]:
pd.options.display.float_format = '{:,.3f}'.format

# 직전3년평균손해율
## TODO: 비비례출재실적 미반영 (반영 여부 추후 검토)
자동차_원수_직전3년연간경과보험료_가공 = 자동차_원수_직전3년연간경과보험료.copy()
자동차_원수_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년연간경과보험료_가공, 자동차_상품군정보)
자동차_원수_직전3년연간경과보험료_집계 = 자동차_원수_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_ELP_PRM']].sum()

자동차_비례출재_직전3년연간경과보험료_가공 = 자동차_비례출재_직전3년연간경과보험료.copy()
자동차_비례출재_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년연간경과보험료_가공, 자동차_상품군정보)
자동차_비례출재_직전3년연간경과보험료_집계 = 자동차_비례출재_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['P_RN_ELP_PRM']].sum()

자동차_원수_직전3년연간손해액_가공 = 자동차_원수_직전3년연간손해액.copy()
자동차_원수_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년연간손해액_가공, 자동차_상품군정보)
자동차_원수_직전3년연간손해액_집계 = 자동차_원수_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_LOSS']].sum()

자동차_비례출재_직전3년연간손해액_가공 = 자동차_비례출재_직전3년연간손해액.copy()
자동차_비례출재_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년연간손해액_가공, 자동차_상품군정보)
자동차_비례출재_직전3년연간손해액_집계 = 자동차_비례출재_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['P_RN_LOSS']].sum()

자동차_직전3년평균손해율 = 자동차_원수_직전3년연간경과보험료_집계 \
    .merge(자동차_비례출재_직전3년연간경과보험료_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(자동차_원수_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(자동차_비례출재_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer')
자동차_직전3년평균손해율[['OGL_ELP_PRM', 'P_RN_ELP_PRM', 'OGL_LOSS', 'P_RN_LOSS']] = 자동차_직전3년평균손해율[['OGL_ELP_PRM', 'P_RN_ELP_PRM', 'OGL_LOSS', 'P_RN_LOSS']].fillna(0)
자동차_직전3년평균손해율.eval('RET_ELP_PRM = OGL_ELP_PRM - P_RN_ELP_PRM', inplace=True)
자동차_직전3년평균손해율.eval('RET_LOSS = OGL_LOSS - P_RN_LOSS', inplace=True)
자동차_직전3년평균손해율.eval('LOSS_RATIO = RET_LOSS/RET_ELP_PRM', inplace=True)
자동차_직전3년평균손해율 = 자동차_직전3년평균손해율.groupby('BOZ_CD', as_index=False)['LOSS_RATIO'].mean()
자동차_직전3년평균손해율

,BOZ_CD,LOSS_RATIO
0,B001,1.771
1,B002,1.548
2,B003,1.577
3,B004,1.503
4,B005,1.919
5,B006,1.528
6,B007,1.558


In [10]:
# pd.options.display.float_format = '{:,.3f}'.format

# # 위험계수
# def get_comb_ratio(data: pd.DataFrame) -> pd.DataFrame:
#     boz_cd_mapper = {'근재': 'A005', '기술':  'A002', '기타':  'A010', '농작물재해보상':  'A009', '상해':  'A007', '외국인상해':  'A008', '종합':  'A003', '책임':  'A006', '해상':  'A004', '화재':  'A001'}
#     return data \
#         .loc[lambda x: x['당월/누계'] == '누계'] \
#         .query('대상구분 == "보유"') \
#         .query('~중분류.str.contains("해외")', engine='python') \
#         .query('~중분류.str.contains("보증")', engine='python') \
#         .assign(중분류 = lambda x: x.중분류.str.replace(r'\d+(.) ', '', regex=True)) \
#         .assign(중분류 = lambda x: x.중분류.str.replace('권원보험', '기타특종')) \
#         .assign(중분류 = lambda x: x.중분류.str.replace('기타특종', '기타')) \
#         .assign(중분류 = lambda x: x.중분류.str.replace('근로자재해보상', '근재')) \
#         .assign(중분류 = lambda x: x.중분류.str.replace('종합보험', '종합')) \
#         .assign(KICS분류 = lambda x: np.where(x.보험종목.isin(['10902', '10903']), '농작물재해보상', np.where(x.보험종목 == '10607', '외국인상해', x.중분류))) \
#         .assign(BOZ_CD = lambda x: x['KICS분류'].apply(lambda y: boz_cd_mapper.get(y, '#'))) \
#         .groupby('BOZ_CD')[['경과보험료', 'IFRS손해액', '순사업비']].sum() \
#         .eval('합산비율 = (IFRS손해액+순사업비)/경과보험료')[['합산비율']]

# 일반_직전1년마감실적_가공 = get_comb_ratio(일반_직전1년마감실적) \
#     .rename(columns={'합산비율': '합산비율_직전1년'}) \
#     .reset_index()

# 일반_1년전1년마감실적_가공 = get_comb_ratio(일반_1년전1년마감실적) \
#     .rename(columns={'합산비율': '합산비율_1년전1년'}) \
#     .reset_index()

# 일반_2년전1년마감실적_가공 = get_comb_ratio(일반_2년전1년마감실적) \
#     .rename(columns={'합산비율': '합산비율_2년전1년'}) \
#     .reset_index()

# 일반_위험계수_가공 = 일반_2년전1년마감실적_가공 \
#     .merge(일반_1년전1년마감실적_가공, on='BOZ_CD', how='outer') \
#     .merge(일반_직전1년마감실적_가공, on='BOZ_CD', how='outer') \
#     .merge(일반_위험계수, on='BOZ_CD', how='outer') \
#     .eval('COMB_RATIO = (합산비율_직전1년 + 합산비율_1년전1년 + 합산비율_2년전1년)/3') \
#     .assign(RSK_COEF_PRM = lambda x: np.fmax(x.BSE_RSK_COEF_PRM*0.7, x.BSE_RSK_COEF_PRM+(x.COMB_RATIO-x.BSE_COMB_RATIO)*0.5)) \
#     [['BOZ_CD', 'COMB_RATIO', 'RSK_COEF_PRM', 'RSK_COEF_RSV']]

In [12]:
pd.options.display.float_format = '{:,.0f}'.format

# 특약보종별 직전1년실적
자동차_특약보종별_직전1년경과보험료_가공 = 자동차_특약보종별_직전1년경과보험료.copy()
자동차_특약보종별_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_특약보종별_직전1년경과보험료_가공, 자동차_상품군정보)
자동차_특약보종별_직전1년경과보험료_가공 = 자동차_특약보종별_직전1년경과보험료_가공.groupby(['BOZ_CD', 'UY'], as_index=False)[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR']].sum()

자동차_특약보종별_직전1년손해액_가공 = 자동차_특약보종별_직전1년손해액.copy()
자동차_특약보종별_직전1년손해액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_특약보종별_직전1년손해액_가공, 자동차_상품군정보)
자동차_특약보종별_직전1년손해액_가공 = 자동차_특약보종별_직전1년손해액_가공.groupby(['BOZ_CD', 'UY'], as_index=False)[['OGL_LOSS_1YR', 'RN_LOSS_1YR']].sum()

자동차_특약보종별_직전1년실적 = 자동차_특약보종별_직전1년경과보험료_가공 \
    .merge(자동차_특약보종별_직전1년손해액_가공, on=['UY', 'BOZ_CD'], how='left')
자동차_특약보종별_직전1년실적[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR', 'OGL_LOSS_1YR', 'RN_LOSS_1YR']] = 자동차_특약보종별_직전1년실적[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR', 'OGL_LOSS_1YR', 'RN_LOSS_1YR']].fillna(0)

In [26]:
자동차_특약보종별_직전1년실적

,BOZ_CD,UY,OGL_ELP_PRM_1YR,RN_ELP_PRM_1YR,OGL_LOSS_1YR,RN_LOSS_1YR
0,B001,2016,"103,983","8,803","1,678,824,902","-806,504,748"
1,B001,2017,"5,041",780,"20,461,600","5,102,866"
2,B002,2016,"286,700","11,582","-391,664,726","-95,398,536"
3,B002,2017,"170,090","25,552","-78,255,692","-10,409,458"
4,B003,2018,"13,316,260","1,804,231","3,267,533,432","916,370,682"
5,B003,2019,"18,630,585,325","3,398,678,288","36,515,288,114","6,219,901,336"
6,B003,2020,"16,394,699,077","2,459,314,128","27,282,128,590","4,058,668,392"
7,B004,2018,"18,086,564","2,510,342","630,189,006","178,860,588"
8,B004,2019,"27,179,402,707","5,338,384,397","47,382,787,746","8,461,107,662"
9,B004,2020,"22,889,757,465","3,433,622,808","40,594,999,398","5,993,047,510"


In [13]:
# 보유리스크율
## 위험계수적용법
자동차_보유리스크율_위험계수적용법 = []
for boz_cd in 자동차_특약보종별_직전1년실적['BOZ_CD'].unique():
        prem = 자동차_특약보종별_직전1년실적_가공.query('BOZ_CD == @boz_cd').groupby('UY', as_index=False)[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR']].sum()
        if len(prem) == 0: continue
        # comm = 자동차_특약수수료_가공.query('TTY_CD_GRP == @tty_cd_grp').drop('TTY_CD_GRP', axis=1)
        loss_ratio = 자동차_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
        # rsk_coef = 자동차_위험계수_가공.query('BOZ_CD == @boz_cd')['RSK_COEF_PRM'].values[0]
        자동차_보유리스크율_위험계수적용법_특약보종별 = get_ret_risk_rate_by_risk_coef(boz_cd, tty_cd_grp, comm, prem, loss_ratio, rsk_coef)
        자동차_보유리스크율_위험계수적용법.append(자동차_보유리스크율_위험계수적용법_특약보종별)
자동차_보유리스크율_위험계수적용법 = pd.concat(자동차_보유리스크율_위험계수적용법, axis=0)
자동차_보유리스크율_위험계수적용법.query('OGL_ELP_PRM_1YR > 0', inplace=True)

NameError: name '일반_특약보종별_직전1년실적_가공' is not defined

In [38]:
pd.options.display.float_format = '{:,.0f}'.format

# 보유리스크율
## 위험계수적용법
일반_보유리스크율_위험계수적용법 = []
for tty_cd_grp in 일반_특약보종별_직전1년실적_가공['TTY_CD_GRP'].unique():
    for boz_cd in 일반_특약보종별_직전1년실적_가공['BOZ_CD'].unique():
        prem = 일반_특약보종별_직전1년실적_가공.query('TTY_CD_GRP == @tty_cd_grp').query('BOZ_CD == @boz_cd').groupby('TTY_YR', as_index=False)[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR']].sum()
        if len(prem) == 0: continue
        comm = 일반_특약수수료_가공.query('TTY_CD_GRP == @tty_cd_grp').drop('TTY_CD_GRP', axis=1)
        loss_ratio = 일반_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
        rsk_coef = 일반_위험계수_가공.query('BOZ_CD == @boz_cd')['RSK_COEF_PRM'].values[0]
        일반_보유리스크율_위험계수적용법_특약보종별 = get_ret_risk_rate_by_risk_coef(boz_cd, tty_cd_grp, comm, prem, loss_ratio, rsk_coef)
        일반_보유리스크율_위험계수적용법.append(일반_보유리스크율_위험계수적용법_특약보종별)
일반_보유리스크율_위험계수적용법 = pd.concat(일반_보유리스크율_위험계수적용법, axis=0)
일반_보유리스크율_위험계수적용법.query('OGL_ELP_PRM_1YR > 0', inplace=True)

## 손해율분포법
일반_보유리스크율_손해율분포법 = []
for tty_cd_grp in 일반_특약보종별_실적_집계['TTY_CD_GRP'].unique():
    for boz_cd in 일반_특약보종별_실적_집계['BOZ_CD'].unique():
        for tty_yr in 일반_특약보종별_실적_집계['TTY_YR'].unique():
            ogl_elp_prm_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['OGL_ELP_PRM'].sum()
            rn_elp_prm_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['RN_ELP_PRM'].sum()
            ogl_loss_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['OGL_LOSS'].sum()
            rn_loss_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['RN_LOSS'].sum()
            ogl_1yr_elp_prm_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['OGL_ELP_PRM_1YR'].sum()
            rn_1yr_elp_prm_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['RN_ELP_PRM_1YR'].sum()
            ogl_1yr_loss_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['OGL_LOSS_1YR'].sum()
            rn_1yr_loss_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['RN_LOSS_1YR'].sum()
            if ogl_1yr_elp_prm_tty_boz == 0: continue
            loss_ratio = 일반_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
            cv = 산업_손해율_변동계수.query('BOZ_CD == @boz_cd')['CV'].values[0]
            slope, a, b, top, bottom = 일반_특약수수료_가공.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')[['CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'TOP_CMSN_RT', 'LWT_CMSN_RT']].values[0]
            ogl_risk, rn_risk = get_ret_risk_rate_by_loss_dist(
                ogl_elp_prm_tty, rn_elp_prm_tty, ogl_loss_tty, rn_loss_tty, 
                ogl_1yr_elp_prm_tty_boz, rn_1yr_elp_prm_tty_boz, ogl_1yr_loss_tty_boz, rn_1yr_loss_tty_boz, 
                loss_ratio, cv, slope, a, b, top, bottom
            )
            일반_보유리스크율_손해율분포법.append([
                boz_cd, tty_yr, tty_cd_grp, ogl_elp_prm_tty, rn_elp_prm_tty, ogl_loss_tty, rn_loss_tty,
                ogl_1yr_elp_prm_tty_boz, rn_1yr_elp_prm_tty_boz, ogl_1yr_loss_tty_boz, rn_1yr_loss_tty_boz,
                loss_ratio, loss_ratio*cv, slope, a, b, top, bottom, ogl_risk, rn_risk
            ])
일반_보유리스크율_손해율분포법 = pd.DataFrame(일반_보유리스크율_손해율분포법, columns=['BOZ_CD', 'TTY_YR', 'TTY_CD_GRP', 'OGL_ELP_PRM_TTY', 'RN_ELP_PRM_TTY', 'OGL_LOSS_TTY', 'RN_LOSS_TTY',
                'OGL_1YR_ELP_PRM_TTY_BOZ', 'RN_1YR_ELP_PRM_TTY_BOZ', 'OGL_1YR_LOSS_TTY_BOZ', 'RN_1YR_LOSS_TTY_BOZ', 
                'LOSS_RATIO_MEAN', 'LOSS_RATIO_STD', 'SLOPE', 'A', 'B', 'TOP', 'BOTTOM', 'OGL_RISK', 'RN_RISK'])
일반_보유리스크율_손해율분포법.query('OGL_1YR_ELP_PRM_TTY_BOZ > 0', inplace=True)

In [112]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/일반_보유리스크율_{now}.xlsx') as writer:
    일반_보유리스크율_위험계수적용법.to_excel(writer, '일반_보유리스크율_위험계수적용법', index=False)
    일반_보유리스크율_손해율분포법.to_excel(writer, '일반_보유리스크율_손해율분포법', index=False)